In [ ]:
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
PERIOD_YEAR = '2014'

In [ ]:
df_interactions = pd.read_csv('../data/userid_trackid_timestamp.tsv', sep='\t')
print(df_interactions.shape)
df_interactions.head()

In [ ]:
df_interactions['year'] = pd.to_datetime(df_interactions['timestamp']).dt.to_period('Y')
df_interactions['month'] = pd.to_datetime(df_interactions['timestamp']).dt.to_period('M')
df_interactions = df_interactions[df_interactions['year'] == PERIOD_YEAR]
print(df_interactions.shape)
df_interactions.head()

In [ ]:
# binarization: we convert the interactions to binary implicit feedback with a threshold of 2 on the interaction counts (reducing false-positive interactions)
def filter_interactions(listening_history: pd.DataFrame, min_interactions: int = 2, verbose: bool = True):
    lhs_count = listening_history.value_counts(subset=['user_id', 'track_id'])
    lhs_count = lhs_count[lhs_count >= min_interactions]
    listening_history = listening_history.set_index(['user_id', 'track_id']).loc[lhs_count.index]
    listening_history = listening_history.reset_index()
    return listening_history

def delete_duplicates(listening_history: pd.DataFrame):
    initial_shape = listening_history.shape
    listening_history = listening_history.drop_duplicates(subset=['user_id', 'track_id'], keep='first')
    print(f"Deleted {initial_shape[0] - listening_history.shape[0]} duplicates")
    return listening_history

df_interactions = filter_interactions(df_interactions, min_interactions=2)
df_interactions = delete_duplicates(df_interactions)
print(df_interactions.shape)
df_interactions.head()

In [ ]:
# sparsity of dataset
n_users = df_interactions['user_id'].nunique()
n_items = df_interactions['track_id'].nunique()
n_interactions = df_interactions.shape[0]
sparsity = 1 - n_interactions / (n_users * n_items)
sparsity

In [ ]:
n_items

In [ ]:
n_users

In [ ]:
# export
df_interactions.to_csv(f'../data/userid_trackid_timestamp_{PERIOD_YEAR}.tsv', sep='\t', index=False)